In [1]:
import os

In [2]:
%pwd

'd:\\IT-service_delivery_risk_predictor\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\IT-service_delivery_risk_predictor'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    processed_data: Path
    reports_dir: Path
    plots_dir: Path
    best_model_path: Path
   

In [7]:
from risk_predictor.constants import *
from risk_predictor.utils.common import read_yaml,create_directories


In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH):
        self.config = read_yaml(config_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        create_directories([config.root_dir, config.reports_dir, config.plots_dir])
        return ModelTrainerConfig(
            root_dir=config.root_dir,
            processed_data=config.processed_data,
            reports_dir=config.reports_dir,
            plots_dir=config.plots_dir,
            best_model_path=config.best_model_path
            
        )

In [9]:
import os, json, joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    accuracy_score,
    roc_curve,
    auc
)
from sklearn.base import clone
from sklearn.preprocessing import label_binarize
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from risk_predictor import logger


In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        os.makedirs(self.config.root_dir, exist_ok=True)
        os.makedirs(self.config.reports_dir, exist_ok=True)
        os.makedirs(self.config.plots_dir, exist_ok=True)
        


        self.models = {
            "Logistic Regression": {
                "model": OneVsRestClassifier(LogisticRegression(max_iter=2000,class_weight='balanced')),
                "params": {
                    "estimator__C": [0.01, 0.1, 1, 10],
                    "estimator__solver": ['lbfgs','saga'],
                    "estimator__penalty": ['l2']
                }
            },     
            "Decision Tree": {
                "model": DecisionTreeClassifier(class_weight='balanced'),
                "params": {
                    'max_depth': [3, 5, 10],
                    'criterion': ['gini', 'entropy']
                }
            },
            "Random Forest": {
                "model": RandomForestClassifier(class_weight='balanced'),
                "params": {
                    'n_estimators': [100, 200],
                    'max_depth': [5, 10, None]
                }
            },
            "XGBoost": {
                "model": XGBClassifier(eval_metric='logloss'),
                "params": {
                    'n_estimators': [100, 200],
                    'learning_rate': [0.05, 0.1],
                    'max_depth': [3, 5]
                }
            },
            "AdaBoost": {
                "model": AdaBoostClassifier(),
                "params": {
                    'n_estimators': [50, 100],
                    'learning_rate': [0.5, 1.0]
                }
            },
            "KNN": {
                "model": KNeighborsClassifier(),
                "params": {
                    'n_neighbors': [3, 5, 7],
                    'weights': ['uniform', 'distance']
                }
            }
        }


    def _save_report_json(self, name, report):
        report_path = Path(self.config.reports_dir) / f"{name}_report.json"
        with open(report_path, "w") as f:
            json.dump(report, f, indent=4)
        logger.info(f"📄 Saved report JSON: {report_path}")

    def _save_report_csv(self, name, report):
        df = pd.DataFrame(report).transpose()
        csv_path = Path(self.config.reports_dir) / f"{name}_report.csv"
        df.to_csv(csv_path, index=True)
        logger.info(f"📊 Saved report CSV: {csv_path}")

    def _save_confusion_matrix(self, name, y_true, y_pred):
        cm = confusion_matrix(y_true, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot(cmap="Blues")
        plt.title(f"Confusion Matrix - {name}")
        cm_path = Path(self.config.plots_dir) / f"{name}_cm.png"
        plt.savefig(cm_path)
        plt.close()
        logger.info(f"📉 Saved confusion matrix plot: {cm_path}")

    def _save_roc_curve(self, name, y_true, y_pred_proba, classes):
        y_true_bin = label_binarize(y_true, classes=classes)
        if y_true_bin.shape[1] == 1:
            return  

        plt.figure()
        for i in range(len(classes)):
            fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, label=f"Class {classes[i]} (AUC={roc_auc:.2f})")

        plt.plot([0, 1], [0, 1], "k--")
        plt.title(f"ROC Curve - {name}")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.legend(loc="lower right")
        roc_path = Path(self.config.plots_dir) / f"{name}_roc.png"
        plt.savefig(roc_path)
        plt.close()
        logger.info(f"📈 Saved ROC curve plot: {roc_path}")

    def train_and_evaluate(self, x_train, y_train, x_test, y_test):
        results = {}
        best_classifier, best_name, best_f1,best_recall,best_precision = None, None, 0, 0, 0

        classes = np.unique(y_train)

        for name, mp in self.models.items():
            logger.info(f"🔹 Training {name}...")

            clf = GridSearchCV(
                mp["model"], mp["params"], 
                cv=5, 
                n_jobs=-1, 
                verbose=1
            )
            clf.fit(x_train, y_train)

            best_model = clone(mp["model"]).set_params(**clf.best_params_)
            best_model.fit(x_train, y_train)

            y_pred = best_model.predict(x_test)
            report = classification_report(y_test, y_pred, output_dict=True)

            # Add best params to the report
            report["best_params"] = clf.best_params_
            

            # Save reports
            self._save_report_json(name, report)
            self._save_report_csv(name, report)

            # Save confusion matrix
            self._save_confusion_matrix(name, y_test, y_pred)

            # ROC curve (if proba available)
            if hasattr(best_model, "predict_proba"):
                y_pred_proba = best_model.predict_proba(x_test)
                self._save_roc_curve(name, y_test, y_pred_proba, classes)

            # Store metrics
            results[name] = {
                "accuracy" :accuracy_score(y_test, y_pred),
                "precision": report['weighted avg']['precision'],
                "recall": report['weighted avg']['recall'],
                "f1": report['weighted avg']['f1-score'],
                "model": best_model
            }

            logger.info(f"{name} → Accuracy={results[name]['accuracy']}, Precision={results[name]['precision']:.2f}, Recall={results[name]['recall']:.2f}, F1={results[name]['f1']:.2f}")

            if results[name]["f1"] < 0.99 :
                if results[name]["f1"] > best_f1:
                        best_f1 = results[name]["f1"]
                        best_precision = results[name]['precision']
                        best_recall = results[name]['recall']
                        best_classifier = best_model
                        best_name = name

        if best_classifier:
            joblib.dump(best_classifier, self.config.best_model_path)
             
             # Save model into best_model_path
            logger.info(f"Best model: {best_name} (F1={best_f1:.2f}, Precision={best_precision:.2f}, Recall={best_recall:.2f}) saved at {self.config.best_model_path}")
        else:
            logger.warning(" No suitable model found!")

        return best_classifier, best_name

In [12]:
try:
        logger.info("Starting Model Training stage")

        # Load configuration
        config = ConfigurationManager()
        model_trainer_config = config.get_model_trainer_config()

        # Initialize ModelTrainer
        model_trainer = ModelTrainer(config=model_trainer_config)

        # Load preprocessed datasets
        X_train = joblib.load("artifacts/data_preprocessing/pickle/X_train.joblib")
        X_test = joblib.load("artifacts/data_preprocessing/pickle/X_test.joblib")
        y_train = joblib.load("artifacts/data_preprocessing/pickle/y_train.joblib")
        y_test = joblib.load("artifacts/data_preprocessing/pickle/y_test.joblib")

        # Execute model training + evaluation (this internally calls _save_report_json, 
        # _save_report_csv, _save_confusion_matrix, and _save_roc_curve for each model)
        best_model, best_name = model_trainer.train_and_evaluate(
            X_train, y_train, X_test, y_test
        )

        if best_model:
            logger.info(f"Model training pipeline completed successfully. Best model: {best_name}")
        else:
            logger.warning("Model training finished but no suitable model found.")

except Exception as e:
    logger.exception("Error occurred during Model Training stage")
    raise e

[2025-10-09 11:09:43,758: INFO: 3719982500: Starting Model Training stage]
[2025-10-09 11:09:43,804: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-09 11:09:43,813: INFO: common: created directory at: artifacts]
[2025-10-09 11:09:43,816: INFO: common: created directory at: artifacts/model_trainer]
[2025-10-09 11:09:43,816: INFO: common: created directory at: artifacts/model_trainer/reports]
[2025-10-09 11:09:43,816: INFO: common: created directory at: artifacts/model_trainer/plots]


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Arya\anaconda3\Lib\logging\__init__.py", line 1163, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\Arya\anaconda3\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f539' in position 44: character maps to <undefined>
Call stack:


[2025-10-09 11:09:43,924: INFO: 1197038668: 🔹 Training Logistic Regression...]


  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\IT-service_delivery_risk_predictor\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\IT-service_delivery_risk_predictor\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\IT-service_delivery_risk_predictor\venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "d:\IT-service_delivery_risk_predictor\venv\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Arya\anaconda3\Lib\asyncio\base_events.py", line 641, in run_forever
    self._run_once()
  File "C:\Users\Arya\anaconda3\Lib\asyncio\base_events.py", line 1986, in _run_once
    handle._run()
  File "C:\Users\Arya\anaconda3\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self.

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[2025-10-09 11:10:15,368: INFO: 1197038668: 📄 Saved report JSON: artifacts\model_trainer\reports\Logistic Regression_report.json]
[2025-10-09 11:10:15,412: INFO: 1197038668: 📊 Saved report CSV: artifacts\model_trainer\reports\Logistic Regression_report.csv]


...

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[2025-10-09 11:15:14,664: INFO: 1197038668: 📄 Saved report JSON: artifacts\model_trainer\reports\KNN_report.json]
[2025-10-09 11:15:14,677: INFO: 1197038668: 📊 Saved report CSV: artifacts\model_trainer\reports\KNN_report.csv]
[2025-10-09 11:15:14,839: INFO: 1197038668: 📉 Saved confusion matrix plot: artifacts\model_trainer\plots\KNN_cm.png]


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Arya\anaconda3\Lib\logging\__init__.py", line 1163, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\Arya\anaconda3\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f4c4' in position 44: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\IT-service_delivery_risk_predictor\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\IT-service_delivery_risk_predictor\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\IT-service_delivery_risk_predictor\venv\Lib\site-packages\ipykernel\kerne

[2025-10-09 11:15:15,313: INFO: 1197038668: 📈 Saved ROC curve plot: artifacts\model_trainer\plots\KNN_roc.png]
[2025-10-09 11:15:15,317: INFO: 1197038668: KNN → Accuracy=0.9097, Precision=0.93, Recall=0.91, F1=0.92]


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Arya\anaconda3\Lib\logging\__init__.py", line 1163, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\Arya\anaconda3\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f4c8' in position 44: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\IT-service_delivery_risk_predictor\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\IT-service_delivery_risk_predictor\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\IT-service_delivery_risk_predictor\venv\Lib\site-packages\ipykernel\kerne

[2025-10-09 11:15:15,581: INFO: 1197038668: Best model: Random Forest (F1=0.98, Precision=0.98, Recall=0.98) saved at artifacts/model_trainer/best_model.joblib]
[2025-10-09 11:15:15,590: INFO: 3719982500: Model training pipeline completed successfully. Best model: Random Forest]
